<a href="https://colab.research.google.com/github/claudiasofiaC/DS-Unit-2-Kaggle-Challenge/blob/master/Copy_of_LS_DS_223_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 2, Module 3*

---

# Cross-Validation


## Assignment
- [ ] [Review requirements for your portfolio project](https://lambdaschool.github.io/ds/unit2), then submit your dataset.
- [ ] Continue to participate in our Kaggle challenge. 
- [ ] Use scikit-learn for hyperparameter optimization with RandomizedSearchCV.
- [ ] Submit your predictions to our Kaggle competition. (Go to our Kaggle InClass competition webpage. Use the blue **Submit Predictions** button to upload your CSV file. Or you can use the Kaggle API to submit your predictions.)
- [ ] Commit your notebook to your fork of the GitHub repo.


You won't be able to just copy from the lesson notebook to this assignment.

- Because the lesson was ***regression***, but the assignment is ***classification.***
- Because the lesson used [TargetEncoder](https://contrib.scikit-learn.org/categorical-encoding/targetencoder.html), which doesn't work as-is for _multi-class_ classification.

So you will have to adapt the example, which is good real-world practice.

1. Use a model for classification, such as [RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)
2. Use hyperparameters that match the classifier, such as `randomforestclassifier__ ...`
3. Use a metric for classification, such as [`scoring='accuracy'`](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values)
4. If you’re doing a multi-class classification problem — such as whether a waterpump is functional, functional needs repair, or nonfunctional — then use a categorical encoding that works for multi-class classification, such as [OrdinalEncoder](https://contrib.scikit-learn.org/categorical-encoding/ordinal.html) (not [TargetEncoder](https://contrib.scikit-learn.org/categorical-encoding/targetencoder.html))



## Stretch Goals

### Reading
- Jake VanderPlas, [Python Data Science Handbook, Chapter 5.3](https://jakevdp.github.io/PythonDataScienceHandbook/05.03-hyperparameters-and-model-validation.html), Hyperparameters and Model Validation
- Jake VanderPlas, [Statistics for Hackers](https://speakerdeck.com/jakevdp/statistics-for-hackers?slide=107)
- Ron Zacharski, [A Programmer's Guide to Data Mining, Chapter 5](http://guidetodatamining.com/chapter5/), 10-fold cross validation
- Sebastian Raschka, [A Basic Pipeline and Grid Search Setup](https://github.com/rasbt/python-machine-learning-book/blob/master/code/bonus/svm_iris_pipeline_and_gridsearch.ipynb)
- Peter Worcester, [A Comparison of Grid Search and Randomized Search Using Scikit Learn](https://blog.usejournal.com/a-comparison-of-grid-search-and-randomized-search-using-scikit-learn-29823179bc85)

### Doing
- Add your own stretch goals!
- Try other [categorical encodings](https://contrib.scikit-learn.org/categorical-encoding/). See the previous assignment notebook for details.
- In additon to `RandomizedSearchCV`, scikit-learn has [`GridSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html). Another library called scikit-optimize has [`BayesSearchCV`](https://scikit-optimize.github.io/notebooks/sklearn-gridsearchcv-replacement.html). Experiment with these alternatives.
- _[Introduction to Machine Learning with Python](http://shop.oreilly.com/product/0636920030515.do)_ discusses options for "Grid-Searching Which Model To Use" in Chapter 6:

> You can even go further in combining GridSearchCV and Pipeline: it is also possible to search over the actual steps being performed in the pipeline (say whether to use StandardScaler or MinMaxScaler). This leads to an even bigger search space and should be considered carefully. Trying all possible solutions is usually not a viable machine learning strategy. However, here is an example comparing a RandomForestClassifier and an SVC ...

The example is shown in [the accompanying notebook](https://github.com/amueller/introduction_to_ml_with_python/blob/master/06-algorithm-chains-and-pipelines.ipynb), code cells 35-37. Could you apply this concept to your own pipelines?


### BONUS: Stacking!

Here's some code you can use to "stack" multiple submissions, which is another form of ensembling:

```python
import pandas as pd

# Filenames of your submissions you want to ensemble
files = ['submission-01.csv', 'submission-02.csv', 'submission-03.csv']

target = 'status_group'
submissions = (pd.read_csv(file)[[target]] for file in files)
ensemble = pd.concat(submissions, axis='columns')
majority_vote = ensemble.mode(axis='columns')[0]

sample_submission = pd.read_csv('sample_submission.csv')
submission = sample_submission.copy()
submission[target] = majority_vote
submission.to_csv('my-ultimate-ensemble-submission.csv', index=False)
```

In [0]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Kaggle-Challenge/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'

In [0]:
import pandas as pd

# Merge train_features.csv & train_labels.csv
train = pd.merge(pd.read_csv(DATA_PATH+'waterpumps/train_features.csv'), 
                 pd.read_csv(DATA_PATH+'waterpumps/train_labels.csv'))

# Read test_features.csv & sample_submission.csv
test = pd.read_csv(DATA_PATH+'waterpumps/test_features.csv')
sample_submission = pd.read_csv(DATA_PATH+'waterpumps/sample_submission.csv')

In [0]:
#imports
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.pipeline import make_pipeline
import category_encoders as ce
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
import matplotlib.pyplot as plt

In [0]:
train.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,status_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,Lake Nyasa,Mnyusi B,Iringa,11,5,Ludewa,Mundindi,109,True,GeoData Consultants Ltd,VWC,Roman,False,1999,gravity,gravity,gravity,vwc,user-group,pay annually,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,functional
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,Lake Victoria,Nyamara,Mara,20,2,Serengeti,Natta,280,NaN,GeoData Consultants Ltd,Other,NaN,True,2010,gravity,gravity,gravity,wug,user-group,never pay,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,Pangani,Majengo,Manyara,21,4,Simanjiro,Ngorika,250,True,GeoData Consultants Ltd,VWC,Nyumba ya mungu pipe scheme,True,2009,gravity,gravity,gravity,vwc,user-group,pay per bucket,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe,functional
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,Ruvuma / Southern Coast,Mahakamani,Mtwara,90,63,Nanyumbu,Nanyumbu,58,True,GeoData Consultants Ltd,VWC,NaN,True,1986,submersible,submersible,submersible,vwc,user-group,never pay,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,non functional
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,Lake Victoria,Kyanyamisa,Kagera,18,1,Karagwe,Nyakasimbi,0,True,GeoData Consultants Ltd,NaN,NaN,True,0,gravity,gravity,gravity,other,other,never pay,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional


In [0]:
# clean data
# make copy 
# copy index and data

def clean_data(x):
  x = x.copy()

  cols_w_zeros = ['longitude', 'latitude', 'construction_year', 
                  'gps_height', 'population']

  for col in cols_w_zeros:
    x[col] = x[col].replace(0, np.nan)    
    x[col+'_missing'] =  x[col].isnull()

  dups = ['quantity_group', 'payment_type']
  x = x.drop(columns=dups)

  u_s = ['recorded_by', 'id']
  x = x.drop(columns=u_s)

  # convert to datetime
  x['date_recorded'] = pd.to_datetime(x['date_recorded'],
                                      infer_datetime_format=True)

  # extract yr, month and day, then drop original
  x['year_recorded'] =   x['date_recorded'].dt.year
  x['month_recorded'] =  x['date_recorded'].dt.month
  x['day_recorded']  =   x['date_recorded'].dt.day 
  x =                    x.drop(columns='date_recorded')

  # new feature, how many years from con_year to date_recorded
  x['years'] = x['year_recorded'] - x['construction_year']
  x['years_missing'] = x['years'].isnull()


  #wrangle time
  return x


In [0]:
train= clean_data(train)
test = clean_data(test)

In [0]:
# define baseline

mean_baseline = train['status_group'].value_counts(normalize=True)

print(f'Mean Baseline\n{mean_baseline}')


Mean Baseline
functional                 0.543081
non functional             0.384242
functional needs repair    0.072677
Name: status_group, dtype: float64


In [0]:
# define target and features

target = 'status_group'
features = train.columns.drop(target)

x_train = train[features]
y_train = train[target]

x_test = test[features]

In [0]:
from scipy.stats import randint, uniform


In [0]:
# pipeline

pipeline = make_pipeline(ce.OrdinalEncoder(),
                        SimpleImputer(),
                        RandomForestClassifier(random_state=42))



In [0]:
# parameters for random search cross val

parameters = {'simpleimputer__strategy': ['mean', 'median'],
    'randomforestclassifier__n_estimators': randint(10, 500),
    'randomforestclassifier__max_depth': [5, 10, 15, 20, 25, 30, 35, 40, 45, 50, None],
    'randomforestclassifier__min_samples_split': range(5, 50),
    'randomforestclassifier__max_features': uniform(0,1)}


# random search cross val using pipeline + parameters

searchCV = RandomizedSearchCV(pipeline,
    param_distributions=parameters,
    n_iter=100,
    cv = 5,
    scoring='accuracy',
    verbose=10,
    return_train_score=True,
    n_jobs=-2)

In [0]:
# fit train data through pipeline
# randomize cross val
# test diff parameters

In [0]:
searchCV.fit(x_train, y_train)

Fitting 5 folds for each of 100 candidates, totalling 500 fits
[CV] randomforestclassifier__max_depth=20, randomforestclassifier__max_features=0.07528517578547167, randomforestclassifier__min_samples_split=41, randomforestclassifier__n_estimators=493, simpleimputer__strategy=median 


[Parallel(n_jobs=-2)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  randomforestclassifier__max_depth=20, randomforestclassifier__max_features=0.07528517578547167, randomforestclassifier__min_samples_split=41, randomforestclassifier__n_estimators=493, simpleimputer__strategy=median, score=(train=0.831, test=0.791), total=  27.1s
[CV] randomforestclassifier__max_depth=20, randomforestclassifier__max_features=0.07528517578547167, randomforestclassifier__min_samples_split=41, randomforestclassifier__n_estimators=493, simpleimputer__strategy=median 


[Parallel(n_jobs=-2)]: Done   1 out of   1 | elapsed:   33.8s remaining:    0.0s


[CV]  randomforestclassifier__max_depth=20, randomforestclassifier__max_features=0.07528517578547167, randomforestclassifier__min_samples_split=41, randomforestclassifier__n_estimators=493, simpleimputer__strategy=median, score=(train=0.831, test=0.786), total=  27.8s
[CV] randomforestclassifier__max_depth=20, randomforestclassifier__max_features=0.07528517578547167, randomforestclassifier__min_samples_split=41, randomforestclassifier__n_estimators=493, simpleimputer__strategy=median 


...

[CV]  randomforestclassifier__max_depth=5, randomforestclassifier__max_features=0.7471919945946901, randomforestclassifier__min_samples_split=26, randomforestclassifier__n_estimators=147, simpleimputer__strategy=mean, score=(train=0.719, test=0.714), total=  25.3s
[CV] randomforestclassifier__max_depth=5, randomforestclassifier__max_features=0.7471919945946901, randomforestclassifier__min_samples_split=26, randomforestclassifier__n_estimators=147, simpleimputer__strategy=mean 


[Parallel(n_jobs=-2)]: Done   8 out of   8 | elapsed:  4.2min remaining:    0.0s


[CV]  randomforestclassifier__max_depth=5, randomforestclassifier__max_features=0.7471919945946901, randomforestclassifier__min_samples_split=26, randomforestclassifier__n_estimators=147, simpleimputer__strategy=mean, score=(train=0.722, test=0.720), total=  25.3s
[CV] randomforestclassifier__max_depth=5, randomforestclassifier__max_features=0.7471919945946901, randomforestclassifier__min_samples_split=26, randomforestclassifier__n_estimators=147, simpleimputer__strategy=mean 


[Parallel(n_jobs=-2)]: Done   9 out of   9 | elapsed:  4.6min remaining:    0.0s


[CV]  randomforestclassifier__max_depth=5, randomforestclassifier__max_features=0.7471919945946901, randomforestclassifier__min_samples_split=26, randomforestclassifier__n_estimators=147, simpleimputer__strategy=mean, score=(train=0.719, test=0.725), total=  25.3s
[CV] randomforestclassifier__max_depth=50, randomforestclassifier__max_features=0.5708596636714643, randomforestclassifier__min_samples_split=11, randomforestclassifier__n_estimators=301, simpleimputer__strategy=median 
[CV]  randomforestclassifier__max_depth=50, randomforestclassifier__max_features=0.5708596636714643, randomforestclassifier__min_samples_split=11, randomforestclassifier__n_estimators=301, simpleimputer__strategy=median, score=(train=0.939, test=0.813), total= 1.9min
[CV] randomforestclassifier__max_depth=50, randomforestclassifier__max_features=0.5708596636714643, randomforestclassifier__min_samples_split=11, randomforestclassifier__n_estimators=301, simpleimputer__strategy=median 
[CV]  randomforestclassifie

In [0]:
print(f'best hyperparameters: {searchCV.best_params_}')
print(f'Validation accuracy: {searchCV.best_score_}')

# print out bet parameters and best score


In [0]:
# make prediction for the test set

n_pipeline = searchCV.best_estimator_
y_pred = n_pipeline.predict(x_test)

In [0]:
submission = sample_submission.copy()
submission['status_group'] = y_pred
submission.to_csv('Claudia S C DS9 sub:3 - Cross Validation.csv', index=False)

In [0]:
from google.colab import files
files.download('Claudia S C DS9 sub:3 - Cross Validation.csv')